In [1]:
import os
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from timeit import default_timer as timer
from IPython.display import clear_output

In [2]:

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []    
    for i in range(len(X) - time_steps):
        clear_output(wait=True)
        print('modeling to keras ',round((i/(len(X) - time_steps))*100,2), ('%'), end='')
        print(' ', round(timer() - start), ' seconds')
        v = X.iloc[i: (i+time_steps), 2:4].to_numpy()
        Xs.append(v)
        ys.append(y.iloc[i+time_steps])
    return np.array(Xs), np.array(ys)

In [4]:
start = timer()
#carregando datasets
print('loading dataset...')
train = pd.read_csv('../datasets/com_concept_drift/sdn_train_unormalized.csv')
test = pd.read_csv('../datasets/com_concept_drift/sdn_test_unormalized.csv')

train = train[train.delay>=0]
test = test[test.delay>=0]
print('load duration: ', round(timer() - start))

loading dataset...
load duration:  0


In [5]:
start = timer()
print('creating window')
TIME_STEPS = 1
X_train,Y_train = create_dataset(train, train.delay, time_steps=TIME_STEPS)
X_test,Y_test = create_dataset(test, test.delay, time_steps=TIME_STEPS)

print('2D to 3D duration: ', round(timer() - start))

modeling to keras  99.4 %  638  seconds


# setting LSTM

In [6]:
print('Init config LSTM')
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units=40,
            input_shape=(X_train.shape[1],X_train.shape[2])
        )
    ))
model.add(keras.layers.Dense(units=40))
model.add(keras.layers.Dense(units=40))
model.add(keras.layers.Dense(units=40))
model.add(keras.layers.Dense(units=40))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))

# compiling

In [7]:
loss ="mse"
optim = tf.keras.optimizers.Adam(
    learning_rate=0.0001)
metrics=["accuracy"]

model.compile(loss=loss, optimizer=optim, 
             metrics=metrics
             )

# training 

In [ ]:
print('Init Train')
start = timer()
history = model.fit(
    X_train, Y_train, 
    epochs=100, 
    batch_size= 10,
    validation_split=0.1,
    shuffle=False,
#     callbacks=[tensorboard_callback]
)
print('trraining duration: ',round(timer() - start))

Init Train
Epoch 1/100
4997/4997 [==============================] - 26s 5ms/step - loss: 2189.8638 - accuracy: 0.8077 - val_loss: 1.0486 - val_accuracy: 0.2574
Epoch 2/100
4997/4997 [==============================] - 24s 5ms/step - loss: 2440.0676 - accuracy: 0.8009 - val_loss: 2.0054 - val_accuracy: 1.0000
Epoch 3/100
4997/4997 [==============================] - 25s 5ms/step - loss: 2246.2544 - accuracy: 0.7909 - val_loss: 3.5046 - val_accuracy: 1.0000
Epoch 4/100
4997/4997 [==============================] - 25s 5ms/step - loss: 2435.2690 - accuracy: 0.7810 - val_loss: 6.7944 - val_accuracy: 1.0000
Epoch 5/100
4997/4997 [==============================] - 24s 5ms/step - loss: 2314.9031 - accuracy: 0.7722 - val_loss: 9.2037 - val_accuracy: 1.0000
Epoch 6/100
4997/4997 [==============================] - 24s 5ms/step - loss: 2399.1304 - accuracy: 0.7573 - val_loss: 14.5477 - val_accuracy: 1.0000
Epoch 7/100
4997/4997 [==============================] - 24s 5ms/step - loss: 2447.3350 - accu

# saving model

In [ ]:
print('Saving Model')
model.save('models/lstm')

# loss training

In [ ]:

fig1 = plt.figure()
ax1 = fig1.add_subplot(1,1,1)
ax1.plot(history.history['loss'], label='train')
ax1.plot(history.history['val_loss'], label='validation')
ax1.legend();

# predicting

In [ ]:
y_pred = model.predict(X_test)

# unormalizing 

In [ ]:
f_columns = ['temperature','label']
scaler1 = StandardScaler().fit(train[f_columns])
scaler2 = StandardScaler().fit(train[f_columns])

scaler1= scaler1.fit(train[f_columns].to_numpy())
scaler2 = scaler2.fit(train[['delay']])


#normalizando test
scaler3 = StandardScaler().fit(test[f_columns])
scaler4 = StandardScaler().fit(test[f_columns])

scaler3 = scaler3.fit(test[f_columns].to_numpy())
scaler4 = scaler4.fit(test[['delay']])


In [ ]:
y_test_inv = scaler4.inverse_transform(Y_test.reshape(1,-1))
y_pred_inv = scaler4.inverse_transform(y_pred)

In [ ]:
fig2 = plt.figure()
a2 = fig2.add_subplot(1,1,1)
a2.plot(y_test_inv.flatten(), marker='.', label='true')
a2.legend();

In [ ]:
fig3 = plt.figure()
a3 = fig3.add_subplot(1,1,1)
a3.plot(y_pred_inv.flatten(),'r',marker='.', label='predicted')
a3.legend();


In [ ]:
fig4 = plt.figure()
a4 = fig4.add_subplot(1,1,1)
a4.plot(y_pred_inv.flatten(),'r',marker='.', label='predicted')
a4.plot(y_test_inv.flatten(), marker='.', label='true')
a4.legend();

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test_inv[0], y_pred_inv[:,0])